In [2]:
import pandas as pd
import numpy as np


In [4]:
matches=pd.read_csv('matches.csv')
deliveries=pd.read_csv('deliveries.csv')


In [5]:
total_score_df=deliveries.groupby(['match_id','inning']).sum()['total_runs'].reset_index()


In [7]:
total_score_df=total_score_df[total_score_df['inning']==1]

matches= matches.merge(total_score_df[['match_id','total_runs']],left_on='id',right_on='match_id')

In [8]:
teams=['Sunrisers Hyderabad', 'Mumbai Indians',
        'Royal Challengers Bangalore',
       'Kolkata Knight Riders',  'Kings XI Punjab',
       'Chennai Super Kings', 'Rajasthan Royals',
        'Delhi Capitals']

In [9]:
matches['team1']=matches['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
matches['team2']=matches['team2'].str.replace('Delhi Daredevils','Delhi Capitals')
matches['team1']=matches['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
matches['team2']=matches['team2'].str.replace('Delhi Daredevils','Sunrisers Hyderabad')

matches=matches[matches['team1'].isin(teams)]
matches=matches[matches['team2'].isin(teams)]

matches=matches[matches['dl_applied']==0]

matches=matches[['match_id','city','winner','total_runs']]

In [10]:
delivery_df=matches.merge(deliveries,on='match_id')

delivery_df=delivery_df[delivery_df['inning']==2]

delivery_df['current_score'] = delivery_df.groupby('match_id')['total_runs_y'].cumsum()

delivery_df['runs_left']=delivery_df['total_runs_x']-delivery_df['current_score']

delivery_df['balls_left']=126-(6*delivery_df['over']+delivery_df['ball'])

In [11]:
delivery_df['player_dismissed']=delivery_df['player_dismissed'].fillna('0')
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].apply(lambda x:x if x == "0" else "1")
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].astype('int')
wickets = delivery_df.groupby('match_id')['player_dismissed'].cumsum()
delivery_df['wickets'] = 10 - wickets

delivery_df['crr'] = (delivery_df['current_score']*6)/(120 - delivery_df['balls_left'])
delivery_df['rrr'] = (delivery_df['runs_left']*6)/( delivery_df['balls_left'])


In [12]:
def result(row):
    return 1 if row['batting_team']==row['winner'] else 0
delivery_df['result'] = delivery_df.apply(result,axis=1)

final_df = delivery_df[['batting_team','bowling_team','city','runs_left','balls_left','wickets','total_runs_x','crr','rrr','result']]

final_df=final_df.sample(final_df.shape[0])

final_df.dropna(inplace=True)
final_df=final_df[final_df['balls_left']!=0]


In [14]:
from sklearn.model_selection import train_test_split
x=final_df.iloc[:,:-1]
y=final_df.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1 )

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

trf = ColumnTransformer([
    ('trf', OneHotEncoder(sparse_output=False, drop='first'), ['batting_team', 'bowling_team', 'city'])
], remainder='passthrough')

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',LogisticRegression(solver='liblinear'))
])
pipe.fit(x_train,y_train)
y_pred = pipe.predict(x_test)

In [18]:
cities=delivery_df['city'].unique()

import pickle
pickle.dump(pipe,open('pipe_new.pkl','wb'))